# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
import numpy as np
import re
import nltk
nltk.download(['punkt','wordnet','stopwords'])

from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import sys

In [4]:
sys.argv

['/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py',
 '-f',
 '/root/.local/share/jupyter/runtime/kernel-aeb1dfc9-5826-4665-a866-9f67a8f4b1c3.json']

In [2]:
# load data from database
engine = create_engine('sqlite:///Hevss_p2_table.db')
df = pd.read_sql_table('Hevss_p2_table', engine)

In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df = df.drop(['child_alone'],axis=1)
# Given value 2 in the related field are neglible so it could be error. Replacing 2 with 1 to consider it a valid response.
# Alternatively, we could have assumed it to be 0 also. In the absence of information I have gone with majority class.
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [5]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    '''
    input: text
    output:
    list of Lemmatizing words
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_place_holder_string = 'urlplaceholder'
    
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, url_place_holder_string)
        
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()

    clean_tokens = [stemmer.stem(w) for w in tokens if w not in stopwords.words('english')]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#setting pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
col_names = list(y.columns.values)

In [12]:
def eval_metrics(real, pred, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    real: array. Array containing the real labels.
    pred: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(real[:, i], pred[:, i])
        precision = precision_score(real[:, i], pred[:, i])
        recall = recall_score(real[:, i], pred[:, i])
        f1 = f1_score(real[:, i], pred[:, i])
        
        metrics.append([f1, precision, recall, accuracy])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['F1', 'Precision', 'Recall', 'Accuracy'])
      
    return data_metrics   

In [13]:
report = eval_metrics(np.array(y_test), y_pred, col_names)
print(report)

                              F1  Precision    Recall  Accuracy
related                 0.873817   0.844512  0.905229  0.801678
request                 0.577746   0.798817  0.452514  0.887109
offer                   0.000000   0.000000  0.000000  0.995042
aid_related             0.659580   0.736964  0.596903  0.749619
medical_help            0.210736   0.654321  0.125592  0.924294
medical_products        0.141892   0.807692  0.077778  0.951564
search_and_rescue       0.233766   0.666667  0.141732  0.977498
security                0.021978   0.333333  0.011364  0.983028
military                0.175824   0.592593  0.103226  0.971396
water                   0.462500   0.787234  0.327434  0.950801
food                    0.703340   0.846336  0.601681  0.942410
shelter                 0.396825   0.781250  0.265957  0.927536
clothing                0.211765   0.750000  0.123288  0.987223
money                   0.108108   0.857143  0.057692  0.981121
missing_people          0.000000   0.000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fde6330d0d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [15]:
parameters = {
    'clf__estimator__min_samples_split': [2, 5],
    'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid=parameters,  n_jobs=4, verbose=2)

In [16]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=10.0min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=10.1min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=10.1min
[CV] clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total=13.0min
[CV] clf__estimator__min_samples_split=5, clf__estimato

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 49.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 49.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__min_samples_split': [2, 5], 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [17]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.24337, std: 0.00822, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 10},
 mean: 0.25119, std: 0.00406, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20},
 mean: 0.23465, std: 0.00248, params: {'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 10},
 mean: 0.24919, std: 0.00540, params: {'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 20}]

In [18]:
#finding the best paramesters based on grip search
print(cv.best_params_)

{'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}


In [19]:
#building new model
optimised_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred = optimised_model.predict(X_test)

In [21]:
report = eval_metrics(np.array(y_test), y_pred, col_names)
print(report)

                              F1  Precision    Recall  Accuracy
related                 0.886913   0.839166  0.940422  0.818078
request                 0.577842   0.820988  0.445810  0.888825
offer                   0.000000   0.000000  0.000000  0.995042
aid_related             0.703082   0.754301  0.658376  0.774027
medical_help            0.179959   0.656716  0.104265  0.923532
medical_products        0.122867   0.782609  0.066667  0.950992
search_and_rescue       0.099291   0.500000  0.055118  0.975782
security                0.021505   0.200000  0.011364  0.982647
military                0.123596   0.478261  0.070968  0.970252
water                   0.534137   0.836478  0.392330  0.955759
food                    0.659184   0.838961  0.542857  0.936308
shelter                 0.452599   0.804348  0.314894  0.931732
clothing                0.292135   0.812500  0.178082  0.987986
money                   0.125000   0.875000  0.067308  0.981312
missing_people          0.000000   0.000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(optimised_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.